In [1]:
from pyworkforce.queuing import ErlangC
from pyworkforce.queuing import MultiErlangC
import os
import pandas as pd
import PySimpleGUI as sg
import math

In [9]:
sg.theme('Dark Blue 3')

def pegar_planilha():
    df_modelo=pd.DataFrame(columns = ['INTERVALO', 'CHAMS_ENTRANTES', 'AHT', 'ESCALADOS', 'NS_CALCULADO'])
    df_modelo.to_excel("modelo.xlsx", index=False)

#função para gerar necessidade:    
def gerar_NS(vol, aht, ope, inter):
    class ErlangC:
        def intensity(self, rate, duration, interval):
            return (rate / interval) * duration

        def erlang_c(self, agents, rate, duration, interval):
            int = self.intensity(rate, duration, interval)
            erlang_b_inv = 1
            for i in range(agents):
                erlang_b_inv = 1 + erlang_b_inv * i / int
            erlang_b  = 1 / erlang_b_inv
            return agents * erlang_b / (agents - int * (1 - erlang_b))

        def service_level(self, agents, rate, duration, target, interval):
            pw = self.erlang_c(agents, rate, duration, interval)
            int = self.intensity(rate, duration, interval)
            return 1 - (pw * math.exp(-(agents - int) * (target / duration)))
        
    ErlangC = ErlangC()
    
    ate = ope
    vol = round(vol,8)
    aht = aht
    
    x = (ErlangC.service_level(agents = math.ceil(ate * (1-0.28)),
                      rate  = vol,
                      duration = aht/60,
                      target = 20/60,
                      interval = 30))
    return x    

#função para gerar o número de PA's necessárias:
def gerar_necessidade(parametros, requirements):
    param_grid = parametros
    erlang = MultiErlangC(param_grid=param_grid, n_jobs=-1)
    required_positions_scenarios = {"service_level": [requirements], "max_occupancy": [0.95]}
    pessoas_nec = erlang.required_positions(required_positions_scenarios)
    return pessoas_nec

#formula capacidade: ((tempo_logado*(1-improdutividade))*ocupação)/aht
#função para gerar a capacidade:
def gerar_capacidade(_aht, _atendentes):
    capacidade = (((1800*(1-0.28))*0.95)/(_aht))*_atendentes #por intervalo dado o numero de atendentes
    #capacidade = {"capacidade": ((23400*(1-0.28))*0.95)/(_aht[i])} por dia por atendente
    return capacidade

def gerar_output(results_NS, results_capacidade, results_raw_positions, results_positions, a, b, c, d, df):
    
    def um(results_NS, results_capacidade, results_raw_positions, results_positions, df):
        df4 = pd.DataFrame(columns=['NS_GERADO'], data=results_NS)
        return df4
    def dois(results_NS, results_capacidade, results_raw_positions, results_positions, df):
        df2 = pd.DataFrame(columns=['RAW_POSITIONS'], data=results_raw_positions)
        df3 = pd.DataFrame(columns=['POSITIONS'], data=results_positions)
        df_final = pd.concat([df2, df3], axis=1)
        return df_final
    def tres(results_NS, results_capacidade, results_raw_positions, results_positions, df):
        df5 = pd.DataFrame(columns=['CAPACIDADE'], data=results_capacidade)
        return df5
    def quatro(results_NS, results_capacidade, results_raw_positions, results_positions, df):
        df2 = pd.DataFrame(columns=['RAW_POSITIONS'], data=results_raw_positions)
        df3 = pd.DataFrame(columns=['POSITIONS'], data=results_positions)
        df4 = pd.DataFrame(columns=['NS_GERADO'], data=results_NS)
        df5 = pd.DataFrame(columns=['CAPACIDADE'], data=results_capacidade)
        df_final = pd.concat([df2, df3, df4, df5], axis=1)
        return df_final
    
    selecoes = {"ns": a,
                "nec": b,
                "cap": c,
                "todas": d}
            
    call_func = {"ns":  um,
                "nec": dois,
                "cap": tres,
                "todas": quatro}
            
    #pega apenas as seleções == True, add em uma lista apenas o que foi selecionado
    t = []
    for i, j in selecoes.items():
        if j == True:
            t.append(i)

    #para cada item na lista "t" que contem apenas as seleções TRUE, procura no dicionário de funções qual deve ser exetutada
    k = 1
    for i in t:
        y = call_func[f'{i}'](results_NS, results_capacidade, results_raw_positions, results_positions, df)
        df_f = pd.concat([df, y], axis=1)
        if k > 1 and k < 3:
            df_f = pd.concat([df, aux, y], axis=1) 
        if k > 2:
            df_f = pd.concat([df, aux2, aux, y], axis=1)
        if k ==1:
            aux2 = y
        aux = y
        k = k + 1
    return df_f
  
#função para gerar output do calculo do NS
def gerans(_volume, _atendentes, _aht, _ns, param_grid, results_NS, results_capacidade, results_raw_positions, results_positions,df, _inter):
    for i in range(len(_volume)):
        vol, aht, ope, inter = _volume[i], _aht[i], _atendentes[i], _inter[i]
        ns = gerar_NS(vol, aht, ope, inter)
        results_NS.append(ns)
    
#função para gerar output do calculo da necessidade
def geranecessidade(_volume, _atendentes, _aht, _ns, param_grid, results_NS, results_capacidade, results_raw_positions, results_positions,df, _inter):
    for i in range(len(_volume)):
        vol, aht, ns = _volume[i], _aht[i]/60, _ns[i]
        param_grid.update({"transactions": [vol], "aht": [aht], "interval": [30], "asa": [20/60], "shrinkage": [0.28]})
        necessidade = gerar_necessidade(param_grid, ns)
        for item in necessidade:
            results_positions.append(item['positions'])
            results_raw_positions.append(item['raw_positions'])

#função para gerar output do cálculo da capacidade
def geracapacidade(_volume, _atendentes, _aht, _ns, param_grid, results_NS, results_capacidade, results_raw_positions, results_positions,df, _inter):
    for i in range(len(_volume)):
        aht, ope = float(_aht[i]), float(_atendentes[i])
        cap = gerar_capacidade(aht, ope)
        results_capacidade.append(cap)
        
def geratodos(_volume, _atendentes, _aht, _ns, param_grid, results_NS, results_capacidade, results_raw_positions, results_positions,df, _inter):
    #gerarNS
    for i in range(len(_volume)):
        vol, aht, ope  = _volume[i], _aht[i]/60, int(_atendentes[i] * (1 - 0.28))
        param_grid.update({"transactions": [vol], "aht": [aht], "interval": [30], "asa": [20/60], "shrinkage": [0.28]})
        niv_serv = gerar_NS(param_grid, ope)
        #print(gerar_NS(param_grid, ope))
        results_NS.append(niv_serv)
    df4 = pd.DataFrame(columns=['NS_GERADO'], data=results_NS)
    
    #gerar necessidade
    for i in range(len(_volume)):
        vol, aht, ns = _volume[i], _aht[i]/60, _ns[i]
        param_grid.update({"transactions": [vol], "aht": [aht], "interval": [30], "asa": [20/60], "shrinkage": [0.28]})
        necessidade = gerar_necessidade(param_grid, ns)
        for item in necessidade:
            results_positions.append(item['positions'])
            results_raw_positions.append(item['raw_positions'])
    df2 = pd.DataFrame(columns=['RAW_POSITIONS'], data=results_raw_positions)
    df3 = pd.DataFrame(columns=['POSITIONS'], data=results_positions)
    
    #gerar capacidade
    for i in range(len(_volume)):
        aht, ope = float(_aht[i]), float(_atendentes[i])
        cap = gerar_capacidade(aht, ope)
        results_capacidade.append(cap)
    df5 = pd.DataFrame(columns=['CAPACIDADE'], data=results_capacidade)
    
    df=pd.concat([df, df2, df3, df4, df5], axis=1)
    
    return df
    
def Menu():
    layout = [
        
        [sg.Text('Selecione o arquivo:')],
        [sg.Text('Tabela Padrao:'), sg.Button('Download')],
        [sg.Text('Arquivo', size=(8, 1)), sg.InputText(key='input_1'), sg.FileBrowse()],
        [sg.Text('Selecione a operação desejada:')],
        [sg.Checkbox('Nivel de Serviço',key='ns'), sg.Checkbox('Necessidade',key='nec'),sg.Checkbox('Capacidade',key='cap'),
         sg.Checkbox('Todas as opções', key='todas')],
        [sg.Button('Ok'), sg.Button('Sair')]
    ]

    window = sg.Window('Calculadora', layout, grab_anywhere=False)
    event, values = window.read()
    window.close()
    return event,values


def main():
    
    while True:  
        event, values = Menu()
        try:
            if event == sg.WIN_CLOSED or event == 'Sair': #fecha se clicar em sair ou no X
                break

            i = 0
            f1 = values['input_1']
            df=pd.read_excel(f1) 
            i = 1
            column_headers = list(df.columns.values)
            
            #preparação de variáveis para execução das funções
            results_positions=[]
            results_raw_positions=[]
            results_NS=[]
            results_capacidade=[]
            param_grid = {"transactions": [0], "aht": [0], "interval": [0], "asa": [0], "shrinkage": [0]}
            
            #adição de variáveis para execução dos cálculos
            df['DEMAND_CALLS'] = df['CHAMS_ENTRANTES'] * df['AHT'] / 600
            _volume = list(df['CHAMS_ENTRANTES'])
            _atendentes = list(df['ESCALADOS'])
            _aht = list(df['AHT'])
            _ns = list(df['NS_CALCULADO'])
            _inter = list(df['INTERVALO'])

            a = values['ns']
            b = values['nec']
            c = values['cap']
            d = values['todas']
            
            # dicionário com as escolhas do usuário
            selecoes = {"ns": values['ns'],
                        "nec": values['nec'],
                        "cap": values['cap'],
                        "todas": values['todas']}

            call_func = {"ns":  gerans,
                         "nec": geranecessidade,
                         "cap": geracapacidade,
                         "todas": geratodos}

            #pega apenas as seleções == True, add em uma lista apenas o que foi selecionado
            t = []
            for i, j in selecoes.items():
                if j == True:
                    t.append(i)
                    
            #para cada item na lista "t" que contem apenas as seleções TRUE, procura no dicionário de funções qual deve ser exetutada
            for i in t:
                call_func[f'{i}'](_volume, _atendentes, _aht, _ns, param_grid, results_NS, results_capacidade, results_raw_positions, results_positions,df, _inter)
            y = gerar_output(results_NS, results_capacidade, results_raw_positions, results_positions, a, b, c, d, df)
            y.to_excel("output.xlsx")
            sg.Popup ('Output gerado!')
                    
        except:
            if event == 'Download':
                sg.Popup ('Fazendo download da tabela padrão. Na coluna intervalo considere 30 min entre cada valor!')
                pegar_planilha()
            elif i == 0:
                sg.popup_error('Arquivo não upado.')
            elif i == 1 and column_headers != ['INTERVALO', 'CHAMS_ENTRANTES', 'AHT', 'ESCALADOS', 'NS_CALCULADO']:                
                sg.popup_error('Arquivo Incorreto, faça o upload da planilha modelo devidamente preenchida.')
            else:
                sg.popup_error('Erro desconhecido.')
                return
        
if __name__ == '__main__':
    main()